In [32]:
# # Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui.
# Taisomas pritaikyti ir mit2zive duomenims
# 
# Skriptas pritaikytas tik Zive duomenims, duomenims mit2zive duomenims skriptas nepritaikytas
# 
# Taisoma - viskas pataisyta:
# - pakeisti ML anotacijų pavadinimus N į Npred, S į Spred, V į Vpred, pridėti Upred
# - pridėti N, S, V, U - čia bus tikrieji
# - pridėti eilutės pabaigoje recordingId, userID
# - pakeisti anotacijų pasiskirstyme anotacijų skaičius į simbolius N, S, V, U
# - anotuoti U klasifikuojant ir skaičiuojant klaidas turi būti praleisti

# Variantas v3 - skirtas atsikratyti iš anksto parengtam failui all_beats_attr. Bandoma jį suformuoti eigoje
# Perdarytas iš 6_zive_accuracy_cnn_vu_v2_test_batch_micro.ipynb
# Skrtas masiniam testavimui - variantas su model_cnn_fda_vu_v1

# Perdarytas iš 6_zive_accuracy_cnn_vu_v2_test_batch.ipynb:
# - pritaikytas variantui, kai klasifikacija izoliuota vienam pūpsniui
# - pakeistas naudojamų požymių sąrašas (pritaikytas vasariniam modeliui 1)
# - naudoja požymių skaičiavimą iš zive_cnn_fda_vu_v3_micro.py (from zive_cnn_fda_vu_v3_micro import predict_cnn_fda_vu_v1),
# kurį gavau iš Povilo 2022 10 05 - faile ReadSeqEKG_2022_10_05.py: get_spike_width funkcija keičiasi,
#  keičiasi ir apply_FDA (perpavardinau į get_beat_features_set_fda_vu_v1)
# 
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šis variantas pritaikytas npy formato zive įrašams, kuriems pakeistas, lyginant su 
# originaliais įrašais, failo vardas iš `file_name` į `SubjCode`, pridedant `userNr`
# prie `file_name`. 
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, funkcijos 
# paimamos iš aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1, testuojami duomenys
# iš db_folder įrašų saugyklos, jame yra ir failas all_beats_attr. 

# Testavimui imami įrašai iš sąrašo SubjCodes, kuris arba paimamas if failo info_create.json,
# arba iš mokymo, validavimo, testavimo sarašų, pvz. train_subjcode_lst.csv. Visiems įrašams iš šių
# sąrašų egzistuoja informacija apie pūpsnius faile all_beats_attr.
 
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota
 
import tensorflow as tf
import pickle
# import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os, json
from pathlib import Path
# from icecream import ic
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

from zive_util_vu import confusion_matrix_modified
from zive_util_vu import cm2df, show_confusion_matrix

from zive_util_vu import create_dir, create_subdir, get_rev_dictionary
from zive_util_vu import runtime, split_SubjCode, get_SubjCode, create_SubjCode

from zive_util_vu import get_beat_attributes, read_rec_attrib
from zive_util_vu import get_userId, read_rec, read_seq
from zive_util_vu import read_rec, read_seq
from zive_util_vu import get_recId, get_symbol_list

from zive_util_vu import zive_read_df_data, get_rec_file_name, get_label_sums, get_rid_off_class_3

from zive_cnn_fda_vu_v3_micro import get_beat_features_fda_vu_v1_vasara

np.set_printoptions(threshold=sys.maxsize)

print(tf.__version__)

# Pastaba: zive_read_file_1ch importuoju iš zive_cnn_fda_vu_v1, nors ji yra ir zive_util_vu.py
# tam, kad atskirti funkcijas, kurios importuojamos skripte zive analysis, nuo tų funkcijų,
# kurios reikalingos tik zive_accuracy_cnn_vu_v1 ir v2. 

def get_beat_features_set_fda_vu_v1_micro(signal, atr_sample, idx_lst):
# Apskaičiuojami užduotų EKG signalo pūpsnių (per idx_lst) požymiai ir iš jų
# suformuojamas požymių dataframe masyvas, pridedant 'label'

    #  all_beats pritaikytas MIT2ZIVE ir ZIVE duomenims
    all_beats = {'N':0,'R':0, 'L':0, 'e':0, 'j':0, 'A':1,'a':1, 'J':1, 'S':1, 'V':2, 'E':2, 'F':3, 'U':3, 'Q':3}

    beat_features_set = pd.DataFrame()
    omit_idx_set = pd.DataFrame()

    # atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    # atr_symbol = df_rpeaks['annotationValue'].to_numpy()
    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    # test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])
    
    # print("\nGet beat features set from signal:")
    for idx in idx_lst:
        beat_features, omit_idx = get_beat_features_fda_vu_v1_vasara(signal, atr_sample, idx)
        # beat_features_set = beat_features_set.append(beat_features, ignore_index=True)
        if omit_idx.empty:
            # beat_features['label'] = test_labels[idx]
            beat_features_set = pd.concat([beat_features_set, beat_features])
        else:
            omit_idx_set = pd.concat([omit_idx_set, omit_idx])

    # Konvertuojame int pozymius į float64
    beat_features_set['RR_l_0'] = beat_features_set['RR_l_0'].astype(float)
    beat_features_set['RR_r_0'] = beat_features_set['RR_r_0'].astype(float)

    return beat_features_set, omit_idx_set


def predict_cnn_fda_vu_v1_micro(signal, atr_sample, atr_symbol, model_dir):
# ************************************* funkcijai ***************************************************************

#  Iėjimo parametrai:   signal, atr_sample, 
#                       modelio ir scaler_train parametrai
#                       
#  Išėjimo parametrai: pred_y, kai kurie neatpažinti : 'U':3

    # Suformuojame indeksų sąrašą. Formuodami sąrašą eliminuojame pirmą ir paskutinį indeksą

    # Naudojamų požymių sąrašas (vasariniam modeliui 1 panaikinau 'RRl/RRr')
    all_features = ['seq_size','RR_l_0', 'RR_r_0', 'RR_r/RR_l', 'wl_side','wr_side',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']
   
    # print("pradėjome predict_cnn_fda_vu_v1")
    # nuskaitome modelio parametrus
    model_path = Path(model_dir, 'best_model_final_2.h5')
    model = tf.keras.models.load_model(model_path)

    # Nuskaitome scaler objectą
    path_scaler = Path(model_dir, 'scaler.pkl')  
    scaler = pickle.load(open(path_scaler,'rb'))

    idx_lst = list(range(1, len(atr_sample)-1))

    # Formuojame iš pūpsnių požymių masyvą
    # data_frame, omitted = apply_FDA_vasara(signal, idx_lst, atr_sample)
    data_frame, omitted = get_beat_features_set_fda_vu_v1_micro(signal, atr_sample, idx_lst)

    data_frame = data_frame.set_index('idx')
    data_frame.columns = data_frame.columns.astype(str)

    # paruošiame požymių masyvą klasifikatoriui
    data_frame_init = data_frame[all_features]
    data_array = scaler.transform(data_frame_init)
    test_x = data_array
    x_test = test_x.reshape((test_x.shape[0], test_x.shape[1], 1))

    # Pūpsnių klasių atpažinimas
    predictions = model.predict(x_test)
    predictions_y = np.argmax(predictions, axis=1)

# Sužymimi neatpažinti pūpsniai - klasė 3:'U'
    pred_y = np.zeros(len(atr_sample), dtype=int)
    pred_y[0] = 3
    pred_y[len(atr_sample)-1] = 3

    if (omitted.empty != True):
        idxs = list(omitted['idx'].astype('int'))
        for i in range(len(idxs)):
            pred_y[idxs[i]] = 3

# Sužymimi atpažinti pūpsniai 
    selected = data_frame.index.astype('int')
    pred_y[selected] = predictions_y

# Atpažinti pūpsniai, kurie anotuoti simboliu `U`, peržymimi į klasę '3'
# https://www.geeksforgeeks.org/python-get-the-indices-of-all-occurrences-of-an-element-in-a-list/
    indices = np.where(atr_symbol == 'U')[0]
    flag = np.size(indices)
    if (flag):
        pred_y[indices] = 3

    indices = np.where(atr_symbol == 'F')[0]
    flag = np.size(indices)
    if (flag):
        pred_y[indices] = 3

    return pred_y


2.8.2


In [40]:
# Pagrindinis skriptas

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Herkulis
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_2022_RUDUO'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v1'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print('Visos galimos anotacijos:', list(all_beats.keys()))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)
# print("\n")

# PASIRUOŠIMAS

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas 
all_features = ['seq_size','RR_l_0', 'RR_r_0', 'RR_r/RR_l','wl_side','wr_side',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']

print(f"Atliekama pūpsnių su anotacijomis {list(selected_beats.keys())} pacientų įrašuose klasifikacija")

# NURODOME PACIENTŲ SĄRAŠĄ. GALIMI ĮVAIRŪS VARIANTAI

# Variantas: visi duomenys
# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)
# SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Variantas: mokymo imtis  
# file_path = Path(rec_dir, 'train_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Variantas: validation imtis  
# file_path = Path(rec_dir, 'validation_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Variantas: testinė imtis  
# file_path = Path(rec_dir, 'test_subjcode_lst.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Pacientų įrašų sąrašas testavimui
# file_path = 'testinis_sarasas.csv'
# SubjCodes = [10020, 10021, 10051] #Testavimui
# SubjCodes = [10022, 10021, 10083, 10091] #Testuojamas pacientų įrašų sąrašas
# SubjCodes = [105411, 10190,10022,10021, 10083, 100, 10091, 10092, 105] #Testavimui
# SubjCodes = [10091, 10092] #Testavimui
# SubjCodes = [10091] #Testavimui
SubjCodes = [105, 10190] #Testavimui
# SubjCodes = [10091,10092,300] #Testavimui
# SubjCodes = [10091,100] #Testavimui
# Variantas: testinė imtis 

# mark1

# file_path = Path(rec_dir, 'test_subjcode_lst_z.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

print("Klasifikuojamų įrašų sąrašas:", SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Suformuojamas aplankas rezultatams

head_tail = os.path.split(file_path)
filename, file_extension = os.path.splitext(head_tail[1])
path_for_results = Path(db_path, 'rezultatai' + '_' + filename)
print(f"\nAplankas rezultatams: {path_for_results}")
create_dir(path_for_results)

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
# print(class_names)

                    # SUFORMUOJAME all_beats_attr

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N', 'S':'S', 'V':'V', 'U':'U'}

# Sukūriame masyvą sekų atributų sąrašui
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

all_beats_attr = pd.DataFrame({'userNr': pd.Series(dtype='int'),
                   'recordingNr': pd.Series(dtype='int'),
                   'sample': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str'),
                   'label': pd.Series(dtype='int'),
                   'pred_label': pd.Series(dtype='int')
                   })

df_rec_results = pd.DataFrame({'userNr':pd.Series(dtype='int'),
    'recordingNr':pd.Series(dtype='int'), 'file_name':pd.Series(dtype='str'),  
    'userId':pd.Series(dtype='str'), 'recId':pd.Series(dtype='int'), 'signal_length':pd.Series(dtype='int'),
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'), 'U':pd.Series(dtype='int'),
    'Nml':pd.Series(dtype='int'), 'Sml':pd.Series(dtype='int'), 'Vml':pd.Series(dtype='int'), 'Uml':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# CIKLAS PER PACIENTŲ ĮRAŠUS

start_time = time.time()

for SubjCode in SubjCodes:
    # print("\nECG įrašas:", SubjCode)
    
    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    userNr, recNr = split_SubjCode(SubjCode)

    # Nuskaitome EKG įrašą (npy formatu)
    sign_raw = read_rec(rec_dir, SubjCode)
    signal_length = sign_raw.shape[0]
    signal = sign_raw
    
    # Surandame ir išvedame įrašo atributus
    userId, recId, file_name = get_recId(rec_dir, userNr, recNr)
    print(f"\nSubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2} userId: {userId} recId: {recId} signal_length: {signal_length}")

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    label_sums, total = get_label_sums(test_labels, all_beats)  
    # print("test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)

    # Surandame ML anotacijų skaitmenines reikšmes pred_labels
    # Jei atr_symbol atranda anotaciją 'U', tai jos neklasifikuoja,
    # bet iš karto patalpina '3' į pred_labels atitinkamą vietą.
    # Į pred_labels taip pat įrašomas '3' pirmam ir paskutiniam pūpsniui,
    # o taip pat pakliuvusiems į ommited sritį, 
    pred_labels = predict_cnn_fda_vu_v1_micro(signal, atr_sample, atr_symbol, model_dir)
    
    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_labels ir pred_labels ilgiai")     

    label_sums_ml, total = get_label_sums(pred_labels, all_beats)  
    # print("pred_labels: ", list(all_beats.keys()), label_sums_ml, "Total:", total)

    # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':test_labels[i], 'pred':pred_labels[i]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    classification = []  # uzblokuota
    if (classification):
        print('\nVietos su ekstrasistolėmis test_y arba pred_y:')
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")  

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):

        # Formuojame pūpsnio atributus
        beats_attr = {'userNr':int(userNr), 'recordingNr':int(recNr), 'sample':int(i_sample), 
                        'symbol':str(atr_symbol[i]), 'label':test_labels[i], 'pred_label':pred_labels[i]}

        # Kaupiame su concat
        df_new_row = pd.DataFrame([beats_attr])
        all_beats_attr = pd.concat([all_beats_attr, df_new_row])

 # Suformuojame klasių numerių masyvus confusion matricai skaičiuoti, surandama confusion matrica

    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_labels ir pred_labels ilgiai")     

    test_labels_mod, pred_labels_mod = get_rid_off_class_3(test_labels, pred_labels)
    label_sums_ml_3, total = get_label_sums(pred_labels_mod, all_beats)  
    # print("pred_labels_3: ", list(all_beats.keys()), label_sums_ml_3, "Total:", total)

    confusion = confusion_matrix(test_labels_mod, pred_labels_mod)
    # print()
    # print(confusion)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_labels_mod, pred_labels_mod, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(label_sums[0]):>5} S:{(int(label_sums[1])):3} V:{int(label_sums[2]):3} U:{int(label_sums[3]):3}" 
    str2 =f"  Nml:{int(label_sums_ml[0]):>5} Sml:{(int(label_sums_ml[1])):3} Vml:{int(label_sums_ml[2]):3} Uml:{int(label_sums_ml[3]):3}" 
    str3 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str4 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str5 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    # print()
    print(str1+str2+str3+str4+str5)

    dict_rec_results = {'userNr':int(userNr), 'recordingNr':recNr, 'file_name':file_name,
    'userId': userId, 'recId': recId, 'signal_length': signal_length,
    'N':label_sums[0], 'S':label_sums[1], 'V':label_sums[2], 'U':label_sums[3],
    'Nml':label_sums_ml[0], 'Sml':label_sums_ml[1], 'Vml':label_sums_ml[2], 'Uml':label_sums_ml[3],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2] 
    }
    rows_list.append(dict_rec_results)

df_rec_results =  pd.DataFrame(rows_list) 

# Ciklo per pacientų įrašus pabaiga

end_time = time.time()
print('\n')
runtime(end_time-start_time)

filepath = Path(path_for_results, 'klasifikacijos_rezultatai_irasams.csv') 
df_rec_results.to_csv(filepath)    
print(f'\nRezultatai įrašyti į:  {filepath}')

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
all_beats_attr.reset_index(inplace = True, drop = True)

# Įrašome sekos atributų masyvą į rec_dir aplanką
file_path = Path(path_for_results, 'all_beats_attr.csv')
all_beats_attr.to_csv(file_path)
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )


Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui
Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio
OS in my system :  win32

Bendras duomenų aplankas:  D:\DI
Zive duomenų aplankas:  DUOM_2022_RUDUO
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  D:\DI\DUOM_2022_RUDUO\records_npy
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3
Visos galimos anotacijos: ['N', 'S', 'V', 'U']
Modelio ir scaler parametrai nuskaitomas iš aplanko:  model_cnn_fda_vu_v1
Atliekama pūpsnių su anotacijomis ['N', 'S', 'V'] pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: [105, 10190]
Sąrašas nuskaitytas iš: D:\DI\DUOM_2022_RUDUO\records_npy\info_create_z.json

Aplankas rezultatams: D:\DI\DUOM_2022_RUDUO\rezultatai_info_create_z
Directory 'D:\DI\DUOM_2022_RUDUO\rezultatai_info_create_z' already exists

SubjCode: 105 userNr: 105 file_name: 105.npy userId: 105 recId: 

In [34]:
# MODELIO TIKSLUMO VERTINIMO IŠ VERTINIMO IMTIES REZULTATAI

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(path_for_results, 'all_beats_attr.csv')
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'recordingNr': int,
                                            'sample': int, 'symbol': str, 'label': int, 'pred_label':int })

# Sukūriame anotuotų ir automatiškai priskirtų klasių visų įrašų pūpsniams sąrašus 
validate_ind_lst = all_beats_attr.index
y_validate = np.array(all_beats_attr['label']).astype('int')
y_predicted = np.array(all_beats_attr['pred_label']).astype('int')

print("\nMODELIO TIKSLUMO VERTINIMO REZULTATAI")
print("Modelis iš aplanko: ", model_dir)

label_sums, total = get_label_sums(y_validate, all_beats)  
print("Pūpsnių klasės: ", list(all_beats.keys()), label_sums, "Total:", total)
print("Klasifikuojamos klasės: ['N', 'S', 'V']")
y_validate_mod, y_predicted_mod = get_rid_off_class_3(y_validate, y_predicted)

# APIBENDRINTI REZULTATAI

print('\nAPIBENDRINTI REZULTATAI\n')

# +++++++++++++++++++++++++++++++++++  čia reiktų įdėtiy_validate, y_predicted valymą nuo 3 klasės

# Skaičiuojame ir išvedame klasifikavimo lentelę
confusion = confusion_matrix(y_validate_mod, y_predicted_mod)
pd.set_option('display.precision',3)
show_confusion_matrix(confusion, class_names)
# print('\n')

print("\nClassification Report\n")
# target_names = [key for (key, value) in selected_beats.items()]

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns",200)
pd.set_option('display.width', 1000)

print(classification_report(y_validate_mod, y_predicted_mod, target_names=class_names, digits=3))
report = classification_report(y_validate_mod, y_predicted_mod, target_names=class_names, output_dict=True)
# output_dictbool, default=False, If True, return output as dict.
df_report = pd.DataFrame(report).transpose()
# https://medium.com/@asmaiya/you-can-something-like-this-84d28e0fd31f

# Įrašome į diską
filepath = Path(path_for_results, 'apibendrinti_rezultatai.csv') 
df_report.to_csv(filepath)    
print(f'\nApibendrinti_rezultatai įrašyti į:  {filepath}')



MODELIO TIKSLUMO VERTINIMO REZULTATAI
Modelis iš aplanko:  model_cnn_fda_vu_v1
Pūpsnių klasės:  ['N', 'S', 'V', 'U'] [3401    0  164  124] Total: 3689
Klasifikuojamos klasės: ['N', 'S', 'V']

APIBENDRINTI REZULTATAI

Confusion Matrix
      N  S    V
N  3348  6   10
S     0  0    0
V     4  1  158


Zero values! Cannot calculate Normalized Confusion Matrix

Classification Report

              precision    recall  f1-score   support

           N      0.999     0.995     0.997      3364
           S      0.000     0.000     0.000         0
           V      0.940     0.969     0.955       163

    accuracy                          0.994      3527
   macro avg      0.646     0.655     0.651      3527
weighted avg      0.996     0.994     0.995      3527


Apibendrinti_rezultatai įrašyti į:  D:\DI\DUOM_2022_RUDUO\rezultatai_info_create_z\apibendrinti_rezultatai.csv


In [35]:
# KLAIDŲ PASISKIRSTYMAS PER PACIENTUS IR JŲ ĮRAŠUS


def collect_noise_locs(rec_dir, all_beats_attr):

    all_beat_indices = all_beats_attr.index

    grouped = all_beats_attr.groupby(['userNr', 'recordingNr'])
    for key, group in grouped:
        # print('\n',key)
        userNr = key[0]
        recordingNr = key[1]

        # SubjCode naudojamas atveju, kai duomenis yra formoje SubjCode.npy, SubjCode.json
        SubjCode = create_SubjCode(userNr, recordingNr)
        filepath = Path(rec_dir, str(SubjCode) + '.json') 
        df_noises = zive_read_df_data(filepath, 'noises')
        noise_arr = np.full(shape=len(group), fill_value=0,  dtype=int)
        
        if (not df_noises.empty):
            idx_noise = 0
            for i, row_i in group.iterrows():
                sample = row_i['sample']
                for j, row_j in df_noises.iterrows():
                    if ((sample > row_j['startIndex']) & (sample < row_j['endIndex'])):
                        noise_arr[idx_noise] = 1
                    else:
                        noise_arr[idx_noise] = 0
                # print(idx_noise, noise_arr[idx_noise], sample)        
                idx_noise += 1
        
        # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
        selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
        index_start = selected_ind[0]
        # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
        
        for i in range(len(noise_arr)):
            all_beats_attr.loc[all_beats_attr.index[index_start+i], 'noise'] = noise_arr[i]

    # print("\nall_beats_attr.groupby(['userNr', 'recordingNr'])['noise'].sum():")
    # print(all_beats_attr.groupby(['userNr', 'recordingNr'])["noise"].sum())

    # print("2: all_beats_attr.head():")
    # print(all_beats_attr.head())
    # print(all_beats_attr.tail())

    # return all_beats_attr
    return all_beats_attr['noise'].values


def get_error(y_test, y_pred):
    # Error in %
    n_errors = 0
    for idx in range(len(y_pred)):
        if (y_test[idx] != y_pred[idx]):
            n_errors += 1
    Err = float(n_errors)/len(y_pred)*100.
    Err = round(Err, 1)
    return Err

# Pasiruošimas

pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 15)
pd.set_option('display.width', 1000)



             # Pagrindinis masyvas: df_seq_errors

# Sukuriame ir užpildome dataframe su sekų parametrais
df_seq_errors = pd.DataFrame(columns= ['idx', 'userNr', 'file_name'])
# df_seq_errors = pd.DataFrame()

rows_list = []
for idx in validate_ind_lst:
# Surandame  userNr, recordingNr, symbol
    userNr, recNr, label, symbol = get_beat_attributes(idx, all_beats_attr)
    seq_attr = {'idx': idx, 'userNr':userNr, 'recordingNr':recNr}
    rows_list.append(seq_attr)

# print("rows_list:")
# print(rows_list[:10])

# Čia įdedame informaciją, ar pūpsnys patenka į triukšmų zoną
noise_arr = collect_noise_locs(rec_dir, all_beats_attr)
# noise_arr = noise_arr[validate_ind_lst]

# print("3: all_beats_attr.head():")
# print(all_beats_attr.head())
# print(all_beats_attr.tail())

# print(type(noise_arr) , noise_arr)

df_seq_errors = pd.DataFrame(rows_list)

df_seq_errors['labels'] = pd.Series(y_validate)
df_seq_errors['preds'] = pd.Series(y_predicted)
zeros_arr = np.zeros( y_predicted.shape[0], dtype=int)
df_seq_errors['errors'] = pd.Series(zeros_arr)  
df_seq_errors.loc[df_seq_errors['labels'] != df_seq_errors['preds'], 'errors'] = 1 
df_seq_errors['noises'] = pd.Series(noise_arr)  

# print("\ndf_seq_errors:")
# print(df_seq_errors.head())
# print(df_seq_errors.tail())
# # print(df_seq_errors.info())


# Rezultatų pasiskirstymas per pacientus
                # Pagrindinis masyvas : df_user_errors

print("\nRezultatų pasiskirstymas per pacientus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_user_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'), 'userId':pd.Series(dtype='str'),  # ??????????????????????
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
    'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Išgauname pacientų vidinę (eilės nr) numeraciją
grouped  = df_seq_errors.groupby(['userNr'])
userNrs = list(grouped.groups.keys())
print(f'Pacientų: {len(userNrs)}\n')

for userNr in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    y_test = df_seq_errors.loc[grouped.groups[userNr]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[userNr]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)
    
    noise_arr = df_seq_errors.loc[grouped.groups[userNr]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

    # Testavimui
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

# +++++++++++++++++++++++++++++++++++  čia reiktų įdėti y_test, y_pred valymą nuo 3 klasės
    label_sums, total = get_label_sums(y_test, all_beats)  
    # print("test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)

    y_test_mod, y_pred_mod = get_rid_off_class_3(y_test, y_pred)
    
    prec,rec,fsc,sup = precision_recall_fscore_support(y_test_mod, y_pred_mod, labels=[0, 1, 2], zero_division=0)
    userId = get_userId(rec_dir, userNr)

    label_sums_ml, total = get_label_sums(y_pred, all_beats)  
    # print("pred_labels: ", list(all_beats.keys()), label_sums_ml, "Total:", total)

    dict_user_errors = {'userNr':userNr,'userId':str(userId),
    'N':label_sums[0], 'S':label_sums[1], 'V':label_sums[2], 'U':label_sums[3],
    'Nml':label_sums_ml[0], 'Sml':label_sums_ml[1], 'Vml':label_sums_ml[2], 'Uml':label_sums_ml[3],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err,  'Noise%':Noise
    }
    rows_list.append(dict_user_errors)

df_user_errors =  pd.DataFrame(rows_list)

# Išvedame suformuotą masyvą
tit1 = f"{'userNr':>6} {'userId':>24}"
tit2 = f"{'N':>8} {'S':>4} {'V':>4} {'U':>4}"
tit3 = f"{'Nml':>8} {'Sml':>4} {'Vml':>4} {'Uml':>4}"
tit4 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit5 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit6 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print('\n')
print(tit1+tit2+tit3+tit4+tit5+tit6)

for idx, row in  df_user_errors.iterrows():
    userId = row['userId']
    if (len(userId)) < 4:
        str1 = f"{int(row['userNr']):>6} {userId:>24}"
    else:
        str1 = f"{int(row['userNr']):>6} {userId:>6}"
    # print(userId, type(userId))
    # mark2
    str2 = f"{int(row['N']):>8} {int(row['S']):4} {int(row['V']):4} {int(row['U']):4}" 
    str3 = f"{int(row['Nml']):>8} {int(row['Sml']):4} {int(row['Vml']):4} {int(row['Uml']):4}" 
    str4 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
    str5 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
    str6 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
    print(str1+str2+str3+str4+str5+str6)

filepath = Path(path_for_results, 'rezultatu_pasiskirstymas_per_pacientus.csv') 
df_user_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per pacientus įrašytas į:  {filepath}')

# print(df_user_errors)




Rezultatų pasiskirstymas per pacientus
Pacientų: 2



userNr                   userId       N    S    V    U     Nml  Sml  Vml  Uml   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
   105                      105    2526    0   41  124    2476    6   49  160    1.00  0.99  1.00    0.00  0.00  0.00    0.82  1.00  0.90      1.9      0.0
  1019 6144c682bd0cc5acb7275368     875    0  123    0     876    1  119    2    1.00  1.00  1.00    0.00  0.00  0.00    0.99  0.96  0.98      0.8    100.0

Rezultatų pasiskirstymas per pacientus įrašytas į:  D:\DI\DUOM_2022_RUDUO\rezultatai_info_create_z\rezultatu_pasiskirstymas_per_pacientus.csv


In [36]:
# Rezultatų pasiskirstymas per pacientus ir jų įrašus
# Skaičiuojama visoms 3 klasėms Precision(tikslumas), Recall (atgaminimas), Fscore (F rodiklis)
# 
# https://towardsdatascience.com/you-dont-always-have-to-loop-through-rows-in-pandas-22a970b347ac
# 
pd.set_option("display.max_rows", 6000)
pd.set_option("display.max_columns", 18)
pd.set_option('display.width', 1000)

# Rezultatų pasiskirstymas per pacientų įrašus
print("\n\nRezultatų pasiskirstymas per pacientų įrašus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_rec_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'),
    'recordingNr':pd.Series(dtype='int'), 'recId':pd.Series(dtype='str'), 'file_name':pd.Series(dtype='str'),  
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'), 'U':pd.Series(dtype='int'),
    'Nml':pd.Series(dtype='int'), 'Sml':pd.Series(dtype='int'), 'Vml':pd.Series(dtype='int'), 'Uml':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Sugrupuojame eilutes pagal 'userId','recordingId', suskaičiuojame, kiek kiekviename įraše
# iš viso yra klasifikuojamų sekų (labels) ir kiek padaryta klaidų (errors).
# Grupavimo objektą paverčiame į normalų dataframe objektą

grouped  = df_seq_errors.groupby(['userNr','recordingNr'])
print(f'Pacientų įrašų: {grouped.ngroups}')
# print(f'{grouped.size()=}')

for key in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    # print(f'\nGroup: {key}\n{df_seq_errors.loc[grouped.groups[key]]}')
    userNr = key[0]
    recordingNr = key[1]
    userId, recId, file_name = get_recId(rec_dir, userNr, recordingNr)
    # file_name = get_filename(rec_dir, create_SubjCode(userNr, recordingNr))
    # userId = get_userId(rec_dir, userNr)

    y_test = df_seq_errors.loc[grouped.groups[key]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[key]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)

    noise_arr = df_seq_errors.loc[grouped.groups[key]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.


    # *********************** Testavimui *******************************************************************
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
    # *********************************************************************************************************

# +++++++++++++++++++++++++++++++++++  čia reiktų įdėti y_test, y_pred valymą nuo 3 klasės
    label_sums, total = get_label_sums(y_test, all_beats)  
    # print("test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)

    y_test_mod, y_pred_mod = get_rid_off_class_3(y_test, y_pred)

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test_mod, y_pred_mod, labels=[0, 1, 2], zero_division=0)
    
    label_sums_ml, total = get_label_sums(y_pred, all_beats)  
    # print("pred_labels: ", list(all_beats.keys()), label_sums_ml, "Total:", total)

    dict_rec_errors = {'userNr':int(userNr), 'recordingNr':recordingNr, 'recId':recId, 'file_name':file_name,
    'N':label_sums[0], 'S':label_sums[1], 'V':label_sums[2], 'U':label_sums[3],
    'Nml':label_sums_ml[0], 'Sml':label_sums_ml[1], 'Vml':label_sums_ml[2], 'Uml':label_sums_ml[3],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }
    rows_list.append(dict_rec_errors)

df_rec_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
grouped  = df_rec_errors.groupby('userNr')
for userNr, group in grouped:
    # print(group.dtypes)
    print("\n")
    userId = get_userId(rec_dir, userNr)
    print(f"{'userNr:'} {userNr} {'userId:'} {userId}" )
    tit1 = f"{'recordingNr':>6}  {'recId':>23} {'file_name':>15}"
    tit2 = f"{'N':>8} {'S':>4} {'V':>4} {'U':>4}"
    tit3 = f"{'Nml':>8} {'Sml':>4} {'Vml':>4} {'Uml':>4}"
    tit4 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
    tit5 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
    tit6 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
    print(tit1+tit2+tit3+tit4+tit5+tit6)

    for idx, row in group.iterrows():
        str1 =f"{int(row['recordingNr']):>6} {str(row['recId']):>29} {str(row['file_name']):>15}"
        str2 = f"{int(row['N']):>8} {int(row['S']):4} {int(row['V']):4} {int(row['U']):4}" 
        str3 =f"{int(row['Nml']):>8} {int(row['Sml']):4} {int(row['Vml']):4} {int(row['Uml']):4}" 
        str4 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
        str5 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
        str6 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
        print(str1+str2+str3+str4+str5+str6)

filepath = Path(path_for_results, 'rezultatu_pasiskirstymas_per_irasus.csv') 
df_rec_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per įrašus įrašytas į:  {filepath}')




Rezultatų pasiskirstymas per pacientų įrašus
Pacientų įrašų: 2


userNr: 105 userId: 105
recordingNr                    recId       file_name       N    S    V    U     Nml  Sml  Vml  Uml   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     0                             0         105.npy    2526    0   41  124    2476    6   49  160    1.00  0.99  1.00    0.00  0.00  0.00    0.82  1.00  0.90      1.9      0.0


userNr: 1019 userId: 6144c682bd0cc5acb7275368
recordingNr                    recId       file_name       N    S    V    U     Nml  Sml  Vml  Uml   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     0      6145fd97bd0cc5ed18275bc8     1631036.053     875    0  123    0     876    1  119    2    1.00  1.00  1.00    0.00  0.00  0.00    0.99  0.96  0.98      0.8    100.0

Rezultatų pasiskirstymas per įrašus įrašytas į:  D:\DI\DUOM_2022_RUDUO\rezultatai_info_create_z\rezultatu_pasiskirstymas_per_irasus.csv


In [37]:
# Sukuriami užduoto ilgio sekų vaizdai ir įrašomi į disko atitinkamus klasėms subfolderius

# Reikalingi parametrai:
# - all_beats_attr, suformuojamas automatiškai iš SubjCodes testinių kodų
# - sets_path - aplankas, kuriame bus aplankas 'saved_images' su vaizdais. Šiuo atveju
# sets_path = rec_dir   
# wl_side, wr_side - analizei naudojami langai


def read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, win_ls, win_rs, win_ls_ext, win_rs_ext):
# Išpjauna užduoto ilgio seką iš mit2zive įrašo ir sukuria jos vaizdą su anotacijomis

# rec_dir - paciento EKG įrašų aplankas
# recordingId - paciento EKG įrašo Id - int
# i_sample - R dantelio, kurio atžvilgiu formuojama seka, indeksas viso EKG įrašo reikšmių masyve - int
# win_ls - klasifikuojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs - klasifikuojamo EKG segmento plotis nuo R pūpsnio (iš dešinės)
# win_ls_ext - vaizduojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs_ext - vaizduojamo EKG segmento plotis už R pūpsnio (iš dešinės) 

    ax = plt.gca()

    sequence, sample, label = read_seq(rec_dir, all_beats_attr, idx, win_ls_ext, win_rs_ext)
    if (sample == None):
        print("klaida!")
        return None
    
    seq_start = sample - win_ls_ext
    seq_end = sample + win_ls_ext

    SubjCode = get_SubjCode(idx, all_beats_attr)

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # # suformuojame anotacijų žymes
    beat_symbols, beat_locs = get_symbol_list(atr_symbol,atr_sample, seq_start, seq_end)

    # deltax ir deltay simbolių pozicijų koregavimui
    min = np.amin(sequence)
    max = np.amax(sequence)
    deltay = (max - min)/20
    deltax = len(sequence)/100

    # suformuojame vaizdą
    x = np.arange(0, len(sequence), 1)
    ax.plot(x, sequence, color="#6c3376", linewidth=2)
    left_mark = win_ls_ext - win_ls
    right_mark = win_ls_ext + win_rs
    ax.axvline(x = left_mark, color = 'b', linestyle = 'dotted')
    ax.axvline(x = right_mark, color = 'b', linestyle = 'dotted')
    for i in range(len(beat_locs)):
        ax.annotate(beat_symbols[i],(beat_locs[i]-deltax,sequence[beat_locs[i]]+deltay))
    ax.set_ylim([min, max+2*deltay])
    
    return(ax)

def get_seq_attributes(all_beats_attr, idx):
    # 'userNr', 'recordingNr', 'sample', 'symbol', 'label'
    row = all_beats_attr.loc[idx]
    return list(row) 

# ////////////////////// Užduodami parametrai /////////////////////

# Bendras aplankas vaizdams
images_folder = 'CNN_VU_vasara'
sets_path = rec_dir

# nurodome, kiek reikšmių naudojama prieš R dantelį ir po
wl_side, wr_side = 100, 100

# užduodame, kiek reikšmių vaizduosime prieš R dantelį ir po
wl_side_ext = 720
wr_side_ext = 720

# Užduodame, kiek sekų vaizdų iš kiekvienos klasės įrašysime į diską
img_max = 10

# /////////////////////////////////////////////////////////////////

print("\nBendras aplankas vaizdams:", images_folder)
print("\nMax sekų vaizdų skaičius iš kiekvienos klasės:", img_max)

if (img_max == 0):
    sys.exit()

# sukuriame bendrą aplanką vaizdams
images_dir = Path(sets_path, 'saved_images', images_folder)
create_dir(images_dir)

# klasių simbolinių vardų sąrašas
class_names = selected_beats.keys()

# sukuriame aplankus sekų vaizdams klasėse
create_subdir(images_dir, class_names)

# Sukuriame vaizdų klasėse skaitiklį
n_classes = len(class_names)
skait = np.zeros((n_classes, n_classes), dtype=int)
# print(skait)

# sukuriame selected_beats reversiją
rev_dict = get_rev_dictionary(selected_beats)

y_test = df_seq_errors['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors['preds'].to_numpy(dtype=int)

y_test_mod, y_pred_mod = get_rid_off_class_3(y_test, y_pred)

# *********************** derinimui *******************************
# acc = accuracy_score(y_test, y_pred)
# print(f"\nAccuracy: {acc:.2f}\n")
# cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
# show_confusion_matrix(cnf_matrix, class_names)
# ******************************************************************

# Ciklas per sekas

icycle = 0 
leng = len(y_pred_mod)

for idx in validate_ind_lst:
    if (icycle >= leng):
        continue

# *********************** derinimui ******************************   
    # row = get_seq_attributes(all_beats_attr, idx)
    # print(idx, row)
# ****************************************************************

    # anotuotos klasės (klasių nr ir simboliniai pažymėjimai)
    label = y_test_mod[icycle]
    label_symb = rev_dict[label]

    # print(f" {idx} label {label} symb {label_symb}")

    # klasifikatoriaus priskirtos klasės (klasių nr ir simboliniai pažymėjimai)
    pred = y_pred_mod[icycle]
    pred_symb = rev_dict[pred]
    icycle +=1

    # print(f" {idx} pred {pred} symb {pred_symb}")

    # patikriname, ar neviršytas skaitiklis, jei viršytas, peršokame
    if (skait[label,pred] >= img_max):
        continue
    else:
        skait[label,pred] += 1

    SubjCode = get_SubjCode(idx, all_beats_attr)
    seq_name = str(SubjCode) + '_' + str(idx)
    
     # 'Išpjauname' užduoto ilgio sekas ir sukuriame jų vaizdus
    fig = plt.figure(facecolor=(1, 1, 1), figsize=(18,3))
    # print("rec_dir =", rec_dir)
    ax = read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, wl_side, wr_side, wl_side_ext, wr_side_ext) 
    if (ax == None):
        print(f'Sekai {idx} negali suformuoti išplėstinio vaizdo')
        plt.close()
        continue

    # suformuosime koreguotą failo vardą
    file_name_orig = get_rec_file_name(rec_dir, SubjCode)
               
    # suformuosime koreguotą failo vardą
    if (file_name_orig == None):
        file_name = seq_name + '_' + 'pred_' + pred_symb + ".png" 
    else:
        file_name = str(file_name_orig) + '_' + seq_name + '_' + 'pred_' + pred_symb + ".png" 
    
    # suformuosime kelią į atitinkamą sub-aplanką
    image_subdir = Path(images_dir, label_symb)
    if (os.path.exists(image_subdir) == False):
        print('Klaida! ', image_subdir,' neegzistuoja')
    file_path = Path(image_subdir, file_name)
    # print('file_name: ',file_name, 'file_path: ', file_path)   

    # Įrašome į atitinkamą anotacijai sub-aplanką 
    ax.set_title(file_name)
    plt.savefig(file_path, bbox_inches='tight', pad_inches = 0.2)
    plt.close()

    # if (icycle >= len(y_pred)):
        # break

# ciklo per validate_ind_lst pabaiga

print("\n")
df = cm2df(skait, class_names)
print(df)

print("\nPabaiga.........")



Bendras aplankas vaizdams: CNN_VU_vasara

Max sekų vaizdų skaičius iš kiekvienos klasės: 10
Directory 'D:\DI\DUOM_2022_RUDUO\records_npy\saved_images\CNN_VU_vasara' already exists
Directory 'D:\DI\DUOM_2022_RUDUO\records_npy\saved_images\CNN_VU_vasara\N' already exists
Directory 'D:\DI\DUOM_2022_RUDUO\records_npy\saved_images\CNN_VU_vasara\S' already exists
Directory 'D:\DI\DUOM_2022_RUDUO\records_npy\saved_images\CNN_VU_vasara\V' already exists
klaida!
Sekai 0 negali suformuoti išplėstinio vaizdo
klaida!
Sekai 1 negali suformuoti išplėstinio vaizdo
klaida!
Sekai 2 negali suformuoti išplėstinio vaizdo
klaida!
Sekai 3 negali suformuoti išplėstinio vaizdo
klaida!
Sekai 4 negali suformuoti išplėstinio vaizdo
klaida!
Sekai 5 negali suformuoti išplėstinio vaizdo


    N  S   V
N  10  6  10
S   0  0   0
V   4  1  10

Pabaiga.........


In [38]:
# Klasifikavimo rodiklių skaičiavimas vienam nurodytam įrašui - įrašas turi būti iš testinio sąrašo

userNr, recordingNr = 300, 0

userId, recId, file_name =  get_recId(rec_dir, userNr, recordingNr)
print("\nZive įrašas:")
print(f"userNr: {userNr}  recordingNr: {recordingNr}")
print(f"\nuserId: {userId}  file_name: {file_name}")

grouped  = df_seq_errors.groupby(['userNr','recordingNr'])
y_test = df_seq_errors.loc[grouped.groups[(userNr,recordingNr)]]['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors.loc[grouped.groups[(userNr,recordingNr)]]['preds'].to_numpy(dtype=int)

print("\nAnotacijų pasiskirstymas įraše:")
label_sums, total = get_label_sums(y_test, all_beats)  
print("test_labels: ", list(all_beats.keys()), label_sums, "Total:", total)
label_sums_ml, total = get_label_sums(y_pred, all_beats)  
print("pred_labels: ", list(all_beats.keys()), label_sums_ml, "Total:", total)

noise_arr = df_seq_errors.loc[grouped.groups[(userNr,recordingNr)]]['noises'].to_numpy(dtype=int)
Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

Err = get_error(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f} Error(%): {Err}\n ")

y_test_mod, y_pred_mod = get_rid_off_class_3(y_test, y_pred)

cnf_matrix = confusion_matrix_modified(y_test_mod, y_pred_mod, n_classes)
show_confusion_matrix(cnf_matrix, class_names)
# https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
# *********************************************************************************************************

prec,rec,fsc,sup = precision_recall_fscore_support(y_test_mod, y_pred_mod, labels=[0, 1, 2], zero_division=0)
row = {
    'N':label_sums[0], 'S':label_sums[1], 'V':label_sums[2], 'U':label_sums[3],
    'Nml':label_sums_ml[0], 'Sml':label_sums_ml[1], 'Vml':label_sums_ml[2], 'Uml':label_sums_ml[3],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }

print("\n")
tit1 = f"{'N':>8} {'S':>4} {'V':>4} {'U':>4}"
tit2 = f"{'Nml':>8} {'Sml':>4} {'Vml':>4} {'Uml':>4}"
tit3 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit4 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit5 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4+tit5)

str1 =f"{int(row['N']):>8} {int(row['S']):4} {int(row['V']):4} {int(row['U']):4}" 
str2 =f"{int(row['Nml']):>8} {int(row['Sml']):4} {int(row['Vml']):4} {int(row['Uml']):4}" 
str3 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
str4 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
str5 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
print(str1+str2+str3+str4+str5)



Zive įrašas:
userNr: 300  recordingNr: 0

userId: 300  file_name: 300.npy


KeyError: (300, 0)